In [ ]:
import msgpack
import pandas
import matplotlib
import difflib

## MessagePack as seen on Zenodo

In [ ]:
with open("./abisko_trials/ctd.msg", "rb") as f:
    raw_bytes = f.read(400)
print(raw_bytes)

In [ ]:
log = msgpack.unpack(open("./abisko_trials/ctd.msg", "rb"))
print(type(log))
df = pandas.DataFrame(log['sensor_ctd.pressure_samples'])

In [ ]:
print(list(log.keys()))
print(len(list(log.keys())))

In [ ]:
print(df)

pressure and time as a dict

## converted msgpack from log file from bihal

In [ ]:
log1 = msgpack.unpack(open("./conv_data/msg/sensor_ctd_test.msg", "rb"))
print(len(log))
print(list(log1.keys()))
print(len(list(log1.keys())))

In [ ]:
with open("./conv_data/msg/sensor_ctd_test.msg", "rb") as f:
    raw_bytes1 = f.read(400)
print(raw_bytes1)

In [ ]:
with open("./abisko_trials/ctd.msg", "rb") as f:
    raw_bytes = f.read(400)
print(raw_bytes)

In [ ]:
hex1 = raw_bytes.hex()
hex2 = raw_bytes1.hex()

diff = difflib.unified_diff(hex1.split(), hex2.split(), fromfile='raw_bytes', tofile='raw_bytes1')
print('\n'.join(diff))

"de" at the beginning signifies that this encoding is a 16-bit map
https://github.com/msgpack/msgpack/blob/master/spec.md

In [ ]:
for i, (b1, b2) in enumerate(zip(raw_bytes, raw_bytes1)):
    if b1 != b2:
        print(f"Byte {i}: {b1} != {b2}")

In [ ]:
for key in log:
    value = log[key]
    print(f"{key}: type={type(value)}, length={len(value) if hasattr(value, '__len__') else 'N/A'}")

In [ ]:
for key in log1:
    value = log[key]
    print(f"{key}: type={type(value)}, length={len(value) if hasattr(value, '__len__') else 'N/A'}")

In [ ]:
import pprint
print('log:')
pprint.pprint(log)
print('log1:')
pprint.pprint(log1)

print('Differences:')
for key in set(log.keys()).union(log1.keys()):
    v1 = log.get(key, '<missing>')
    v2 = log1.get(key, '<missing>')
    if v1 != v2:
        print(f'{key}:', v1, '!=', v2)

In [ ]:
def compare_dicts(d1, d2, prefix=''):
    for key in set(d1.keys()).union(d2.keys()):
        p = f'{prefix}{key}'
        if key not in d1:
            print(f'Missing in log: {p}')
        elif key not in d2:
            print(f'Missing in log1: {p}')
        else:
            v1, v2 = d1[key], d2[key]
            if isinstance(v1, dict) and isinstance(v2, dict):
                compare_dicts(v1, v2, prefix=p + '.')
            elif isinstance(v1, list) and isinstance(v2, list):
                if len(v1) != len(v2):
                    print(f'List length differs at {p}: {len(v1)} != {len(v2)}')
                for i, (item1, item2) in enumerate(zip(v1, v2)):
                    if item1 != item2:
                        print(f'Difference at {p}[{i}]: {item1} != {item2}')
            elif v1 != v2:
                print(f'Difference at {p}: {v1} != {v2}')

# Usage:
compare_dicts(log, log1)

In [ ]:
print(list(log.keys()))
print(list(log1.keys()))

# If you see a key like 'sensor_ctd.pressure_samples', print its type and length:
if 'sensor_ctd.pressure_samples' in log:
    print(type(log['sensor_ctd.pressure_samples']))
    print(len(log['sensor_ctd.pressure_samples']))
else:
    print("'sensor_ctd.pressure_samples' not found in log")

In [ ]:
print('Sorted keys in log:')
print(sorted(log.keys()))
print('Sorted keys in log1:')
print(sorted(log1.keys()))

In [ ]:
for key in sorted(set(log.keys()).union(log1.keys())):
    v1 = log.get(key, '<missing>')
    v2 = log1.get(key, '<missing>')
    if type(v1) != type(v2):
        print(f"{key}: type differs ({type(v1)} vs {type(v2)})")
    elif isinstance(v1, dict):
        print(f"{key}: dict, lengths: {len(v1)} vs {len(v2)}, equal: {v1 == v2}")
    elif isinstance(v1, list):
        print(f"{key}: list, lengths: {len(v1)} vs {len(v2)}, equal: {v1 == v2}")
    else:
        print(f"{key}: {v1 == v2}")

In [ ]:
depth1 = log['sensor_ctd.depth_samples']
depth2 = log1['sensor_ctd.depth_samples']

for i, (v1, v2) in enumerate(zip(depth1, depth2)):
    if v1 != v2:
        print(f"First mismatch at index {i}:")
        print("  log :", v1)
        print("  log1:", v2)
        break
else:
    print("No mismatches found (unexpected, since lists are not equal).")

In [ ]:
import math

def safe_equal(a, b):
    if isinstance(a, float) and isinstance(b, float):
        if math.isnan(a) and math.isnan(b):
            return True
        return a == b
    if isinstance(a, dict) and isinstance(b, dict):
        if set(a.keys()) != set(b.keys()):
            return False
        return all(safe_equal(a[k], b[k]) for k in a)
    if isinstance(a, list) and isinstance(b, list):
        if len(a) != len(b):
            return False
        return all(safe_equal(x, y) for x, y in zip(a, b))
    return a == b

for i, (v1, v2) in enumerate(zip(depth1, depth2)):
    if not safe_equal(v1, v2):
        print(f"First real mismatch at index {i}:")
        print("  log :", v1)
        print("  log1:", v2)
        break
else:
    print("No real mismatches found (all differences are just nan vs nan).")